# CDC estimated inpatient beds time series

* [Estimated inpatient beds](https://healthdata.gov/sites/default/files/estimated_inpatient_all_20210221_1631.csv)
* [Estimated inpatient beds in use by COVID-19 patients](https://healthdata.gov/sites/default/files/estimated_inpatient_covid_20210221_1631.csv)
* [Estimated ICU beds](https://healthdata.gov/sites/default/files/estimated_icu_20210221_1631.csv)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
# Get today's path

y, m, d = datetime.now().year, datetime.now().month, datetime.now().day

root_url = "https://healthdata.gov/sites/default/files/estimated"
inpatient_beds_url = f"{root_url}_inpatient_all_{y}{m:02d}{d-1:02d}_1631.csv"
covid_19_inpatient_beds_url = f"{root_url}_inpatient_covid_{y}{m:02d}{d-1:02d}_1631.csv"
occupied_icu_beds_url = f"{root_url}_icu_{y}{m:02d}{d-1:02d}_1631.csv"

In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]

In [ ]:
def grab_and_augment_table(url: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(url)
        df["ISO3166_1"] = "US"
        df["ISO3166_2"] = df["state"]
        df["Last_Reported_Flag"] = df["collection_date"].max() == df["collection_date"]
        return df[df["ISO3166_2"].isin(states)]
    except Exception as e:
        pass

In [ ]:
inpatient_beds = grab_and_augment_table(inpatient_beds_url)
covid_19_inpatient_beds = grab_and_augment_table(covid_19_inpatient_beds_url)
occupied_icu_beds = grab_and_augment_table(occupied_icu_beds_url)

In [ ]:
inpatient_beds.columns = ["STATE", "DATE", "INPATIENT_BEDS_OCCUPIED", "INPATIENT_BEDS_LOWER_BOUND", "INPATIENT_BEDS_UPPER_BOUND", "INPATIENT_BEDS_IN_USE_PCT", "INPATIENT_BEDS_IN_USE_PCT_LOWER_BOUND", "INPATIENT_BEDS_IN_USE_PCT_UPPER_BOUND", "TOTAL_INPATIENT_BEDS", "TOTAL_INPATIENT_BEDS_LOWER_BOUND", "TOTAL_INPATIENT_BEDS_UPPER_BOUND", "ISO3166_1", "ISO3166_2", "LAST_REPORTED_FLAG"]
covid_19_inpatient_beds.columns = inpatient_beds.columns
occupied_icu_beds.columns = ["STATE", "DATE", "STAFFED_ADULT_ICU_BEDS_OCCUPIED", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_LOWER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_UPPER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_LOWER_BOUND", "STAFFED_ADULT_ICU_BEDS_OCCUPIED_PCT_UPPER_BOUND", "TOTAL_STAFFED_ICU_BEDS", "TOTAL_STAFFED_ICU_BEDS_LOWER_BOUND", "TOTAL_STAFFED_ICU_BEDS_UPPER_BOUND", "ISO3166_1", "ISO3166_2", "LAST_REPORTED_FLAG"]

In [ ]:
inpatient_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_ALL.csv", index=False, columns=inpatient_beds.columns)
covid_19_inpatient_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_COVID_19.csv", index=False, columns=covid_19_inpatient_beds.columns)
occupied_icu_beds.to_csv(output_folder + "CDC_INPATIENT_BEDS_ICU_ALL.csv", index=False, columns=occupied_icu_beds.columns)

